# The Open University / Space Science course S818 / TMA03 Question n°3
Student : Madeline Pennerath / L3756198

This Jupyter notebook produces a graph of integrated proton flux above 10MeV as a function of time and for a month to show if a Solar Radiation Storm event occurred. The calculations use data from a GOES satellite that are taken from the National Oceanic and Atmospheric Administration's (NOAA) website (see links below).

In [1]:
# this code conceals irrelevant warning messages
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
# routines:
from pandas import *
from numpy import  *
%matplotlib inline

In [17]:
df = read_csv ('alerts.csv')

In [4]:
#skip the initial spaces after a comma in the columns names
df = read_csv('alerts.csv', skipinitialspace=True)

In [6]:
#Converting the ‘Date’ column in the dataframe into ‘datetime’ format
df['Date'] = to_datetime(df['Date'])

In [7]:
#Teide Observatory 
#latitude 28° 18´ 08´´ (N)
#longitude of –16° 30´ 36´´ (W)
# 01:00 UT on a night between May 21st and May 24th (the telescope eventually took pictures on May 29th without changing these values again)
#local sidereal time varies between
#15h50min30sec and 16h02h19
#Mean value is 15h55min55sec
hourOpt = 15
minOpt = 55
secOpt = 54.8

In [8]:
from math import log10, floor
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

PI_OVER_12  =  pi / 12.0
PI_OVER_12_OVER_60 = pi/(12*60)
PI_OVER_12_OVER_60_OVER_60 = pi/(12*60*60)

def hoursToRadians ( hours ):
    """Convert hours (15 degrees) to radians.
    """
    return  hours * PI_OVER_12

def minutesToRadians ( minutes ):
    """Convert minutes (1/60 hour) to radians.
    """
    return  minOpt * PI_OVER_12_OVER_60

def secondsToRadians ( seconds ):
    """Convert seconds (1/60 minute) to radians.
    """
    return  secOpt * PI_OVER_12_OVER_60_OVER_60

hourOpt = round_sig(hoursToRadians(hourOpt),4)
#hourOpt = hoursToRadians(hourOpt)
minOpt = round_sig(minutesToRadians(minOpt),5)
secOpt = round_sig(secondsToRadians(secOpt),5)

(3.927, 0.23998, 0.0039852)

In [9]:
RA2 = (hourOpt + minOpt + secOpt)
#df['RaDeg'].head()
Dec2 = np.deg2rad(28) #rad

(4.1709651999999995, 0.4886921905584123)

In [11]:
#df['RaDeg'] = np.deg2rad(df['RaDeg'])
#df['DecDeg'] = np.deg2rad(df['DecDeg'])
df['RaDeg'].head()
#enlever Deg dans les noms
#arrondir a cb ?

0    274.56022
1    140.88042
2    308.74244
3    156.13568
4    145.26636
Name: RaDeg, dtype: float64

In [12]:
# cos (A) = sin (Dec1) sin (Dec2) + cos (Dec1) cos (Dec2) cos (RA1 – RA2)
# (RA2, Dec2) is the optimum sky

In [13]:
RaColumn = df['RaDeg']
DecColumn = df['DecDeg']
cosAColumn = sin(np.deg2rad(DecColumn))*sin(Dec2)+cos(np.deg2rad(DecColumn))*cos(Dec2)*cos(np.deg2rad(RaColumn)-RA2)

Acolumn = (np.arccos(cosAColumn))*(180/pi)
df['Offset'] = Acolumn
df['Offset'].head()



0    98.805792
1    77.446525
2    99.926873
3    75.530088
4    77.261290
Name: Offset, dtype: float64

In [14]:
dates = df['Date']
mag = df['AlertMag']
histmag = df['HistoricMag']
Class = df['Class']
start = datetime(2020, 4,24)
end = datetime(2020, 5, 26)


In [15]:
df.dtypes

#Name                     object
Date              datetime64[ns]
RaDeg                    float64
DecDeg                   float64
AlertMag                 float64
HistoricMag              float64
HistoricStdDev           float64
Class                     object
Published                 object
Comment                   object
TNSid                     object
Offset                   float64
dtype: object

In [16]:
df = df[(dates >= start) & (Class != "unknown") & (mag <= 18) & (histmag <= 18) & (Acolumn >= -60) & (Acolumn <= 60)]
df.sort_values("Offset", inplace = True)#, axis = 0, ascending = True, inplace = True)
df

,#Name,Date,RaDeg,DecDeg,AlertMag,HistoricMag,HistoricStdDev,Class,Published,Comment,TNSid,Offset
213,Gaia20cag,2020-04-27 16:43:06,246.25733,39.15731,13.54,16.51,1.52,CV,2020-04-29 10:27:46,known CV V844 Her in 4 mag outburst,AT2020ilw,12.687488
101,Gaia20ceo,2020-05-08 08:53:43,205.54987,60.25699,17.65,17.97,0.08,QSO,2020-05-10 11:40:33,~0.3 mag rise in known QSO,AT2020jkn,39.358046
